# Lugares turísticos, separados por categorías

#### Dada una url de una página de restaurantguru, la inspecciona y extrae las caracteriísticas más relevantes del establecimiento.
- Nombre
- Localidad
- Coordenadas (latitud y longitud)
- Características (dependen del tipo de lugar turístico, pueden ser de un tipo o de otro) 
- Descripción 

In [1]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.options import Options
import re
import numpy as np
import json
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import os

# Funciones

Función 1: extraer, para una url (como objeto response) de TurismoAsturias general (en nuestro caso será referente a una página de un tipo de sitio), las urls de los distintos lugares turísticos que se encuentran en esa página. (Aquí no hay valoración, no aparece en la página de TurismoAsturias)

In [2]:
def obtener_enlaces_pagina_requests(response):
    """
    Función para obtener el total de enlaces de lugares turísticos desde una página cargada de turismoasturias.es.

    Parámetros:
    - response: objeto que contiene el html de la página cargada.

    Salida:
    - Una lista con los enlaces correspondientes.
    """

    # Analiza el contenido html de la página
    soup = BeautifulSoup(response.text, "html.parser")

    # Buscar todos los elementos de la página que tienen enlaces a lugares turísticos
    elementos = soup.find_all(class_='d-block image-wrapper aspect-ratio-bg-cover aspect-ratio-4-to-3')

    # Lista para almacenar las URLs
    enlaces_pagina = []

    # Para cada elemento, se selecciona la URL correspondiente y se agrega a la lista
    for enlace in elementos:
        if enlace.has_attr("href"):  # Verifica si tiene href
            enlaces_pagina.append(enlace["href"])  # Agrega el enlace a la lista
    
    return enlaces_pagina

Función 2: extraer, a partir de una url base, todas los enlaces contenidos en las páginas correspondientes a esa url. Se basa en la función obtener_enlaces_pagina_requests, que para cada página con la url base dada, extrae los enlaces a los diferentes luagres turísticos (requiere conocer la cantidad de páginas de un determinado tipo que existen)

In [3]:
def obtener_enlaces_lugturistico_requests(url_base, pag_final, pag_inicio=1):
    """
    Función para obtener el total de enlaces de lugares turísticos desde una url general con varias páginas que comparten estructura..

    Parámetros:
    - url_base: parte del enlace compartida por todas las páginas.
    - pag_inicio: identificador de la página inicial que es necesario añadir a la url_base (por defecto es 1).
    - pag_final: identificador de la página final

    Salida:
    - Una lista con los enlaces correspondientes.
    """

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    enlaces_completo = []

    # Recorrer las páginas y extraer de cada una de ellas los enlaces disponibles
    for id in range(pag_inicio, pag_final + 1):
        response = requests.get(f"{url_base}{id}", headers=headers) # Cargar el contenido de la página
        enlaces_completo.extend(obtener_enlaces_pagina_requests(response))

    return enlaces_completo

In [4]:
def lug_turistico_requests(response):
    """
    Función para obtener la información relevante de un determinado lugar turístico a partir de una URL de turismoasturias.es usando requests.

    Parámetros:
    - response: objeto que contiene el html de la página cargada.

    Salida:
    - nombre: nombre del lugar turístico.
    - localidad: localidad (ciudad, pueblo, etc.).
    - latitud, longitud: coordenadas del lugar turístico.
    - caracteristicas: características del lugar turístico.
    - descripcion: descripción del lugar turístico.
    """

    # Analiza el contenido html de la página
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Obtener el NOMBRE del lugar turístico
    nombre = soup.find(class_='article-title').get_text(strip=True)
    
    # Obtener la LOCALIDAD
    localidad = soup.find(class_='important').get_text(strip=True)

    # Obtener las COORDENADAS (si están presentes)
    latitud, longitud = "", ""
    gps_span = soup.find("span", {"itemprop": "geo"})

    if gps_span:
        coordenadas = gps_span.get_text(strip=True)
        if "," in coordenadas:
            latitud, longitud = coordenadas.split(",")

    # Obtener las CARACTERÍSTICAS del lugar turístico (eliminar el primer caracter porque es la localidad)
    caracteristicas = [elem.get_text(strip=True) for elem in soup.find_all(class_='field-wrapper')]
    caracteristicas.pop(0)

    # Obtener la DESCRIPCIÓN del lugar turístico (si está)
    descripcion = ""
    descripcion_tag = soup.find(class_='article-section-content article-section-info accordion-collapse collapse show') or \
                     soup.find(class_='article-section-content article-section-color article-section-background accordion-collapse collapse show')
    if descripcion_tag:
        descripcion = descripcion_tag.get_text(strip=True)

    return nombre, localidad, latitud, longitud, caracteristicas,  descripcion # encabezado,


In [5]:
def obtener_datos_enlaces(enlaces, tipo):
    """
    Obtiene información de un conjunto de urls que coinciden en el tipo de lugar turístico y la transforma en dataframe

    Parámetros:
    - enlaces: Lista de URLs de los lugares turisticos.
    - tipo: tipo de lugar turistico correspondiente (ruta, cultura, playa, ...)

    Salida:
    - DataFrame con la información extraída.
    """

    datos = []  # Lista para almacenar los datos
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    # Recorrer todas las URLs de los lugares turísticos
    for url in enlaces:
        response = requests.get(url, headers=headers)  # Hacer la solicitud a la página web
        if response.status_code != 200:
            print(f"Error al obtener la página: {url}")  # Salir de la iteración si hay error
            continue

        # Extraer los datos usando la función lug_turistico_requests
        try: 
            nombre, localidad, latitud, longitud, caracteristicas, descripcion = lug_turistico_requests(response)
            # Guardar los datos en la lista
            datos.append({
                'Nombre': nombre, 'Localización': localidad, 'Latitud': latitud, 'Longitud': longitud,
                "Características": caracteristicas, "Descripción": descripcion, "Tipo_lugar_turistico": tipo , "url": url
            })
        except:
            print("Error en: ", url) # Avisa de la página en la que hay algun error (por seguridad)
            continue

    # Transformar los datos en un DataFrame
    df_resultado = pd.DataFrame(datos)
    return df_resultado

## ESPACIOS CULTURALES

Dada la url de la página correspondiente a los espacios culturales de turismoasturias.es, se busca obtener un dataframe con las características de cada uno de ellos. El primer paso es obtener las urls de cada uno de los espacios culturales existentes con ayuda de la función obtener_enlaces_lugturistico_requests (150 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [6]:
# Extraer todas las url de los museos y espacios culturales
start= time.time() # Contabilizar el tiempo
# Definir los parámetros de la función (url base y páginas de 1 a 13)
url_base = "https://www.turismoasturias.es/cultura/museos-espacios-culturales?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u_delta=12&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u_cur="
pag_final = 13

# Aplicar la función para obtener los enlaces
enlaces_completo = obtener_enlaces_lugturistico_requests(url_base, pag_final)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.35min


In [9]:
# Extraer las características de los enlaces
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Cultura")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 1.79min


In [11]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
143,Pinacoteca Municipal de Langreo - Eduardo Úrculo,Langreo,43.300743,-5.686654,"[Dirección:C/ La Unión, 31 ■33900 - Llangréu/L...","La pinacoteca, ubicada en el antiguo macelo de...",Cultura,https://www.turismoasturias.es/descubre/cultur...
120,Museo Etnográfico de Grandas de Salime 'Pepe e...,Grandas de Salime,43.217552,-6.874403,"[Dirección:Avda. del Ferreiro, 17 ■33730 - Gra...",El Museo fue fundado en el año 1984 sobre la b...,Cultura,https://www.turismoasturias.es/descubre/cultur...
94,Museo de Arte Sacro de Tineo,Tineo,43.335012,-6.415056,[Dirección:Iglesia Parroquial de San Pedro ■33...,Este Museo se ubica dentro del recinto del Con...,Cultura,https://www.turismoasturias.es/descubre/cultur...
97,Museo de la Apicultura,Caso,43.203777,-5.395626,[Dirección:Antigua Escuela (junto a la Iglesia...,"En este concejo, que ocupa una buena parte del...",Cultura,https://www.turismoasturias.es/descubre/cultur...
140,Parque de la Prehistoria de Teverga,Teverga,+43.137829,-6.07588,"[Dirección:■33111 - San Salvador D'Alesga, Tel...",El Parque de la Prehistoria de Teverga es un p...,Cultura,https://www.turismoasturias.es/descubre/cultur...


In [12]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/cultural_spaces.csv', index=False)

## RUTAS

Dada la url de la página correspondiente a las rutas de turismoasturias.es, se busca obtener un dataframe con las características de cada una de ellas. El primer paso es obtener las urls de cada una de las rutas existentes con ayuda de la función obtener_enlaces_lugturistico_requests (120 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [13]:
# Extraer todas las url de las rutas
start= time.time() # Contabilizar el tiempo
# Definir los parámetros de la función (url base y páginas de 1 a 6)
url_base = "https://www.turismoasturias.es/naturaleza/rutas-de-senderismo?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_y8slAeWJKxZf&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_y8slAeWJKxZf_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_y8slAeWJKxZf_cur="
pag_final = 6

# Aplicar la función para obtener los enlaces
enlaces_completo = obtener_enlaces_lugturistico_requests(url_base, pag_final)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.21min


In [15]:
# Extraer las características de los enlaces
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Ruta")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 1.82min


In [19]:
df_resultados.head(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,Acantilados de Pimiango,Ribadedeva,43.39288410991442,-4.527212476016342,"[Distancia:8,1kms, Dificultad:Baja, Altitud:61...",En el límite más oriental de Asturias nos enco...,Ruta,https://www.turismoasturias.es/descubre/natura...
1,Acantilados del Infierno,Ribadesella,43.45941330453343,-5.005828477654885,"[Distancia:5,6kms, Dificultad:Baja, Altitud:73...",En cualquier punto de esta ruta disfrutaremos ...,Ruta,https://www.turismoasturias.es/descubre/natura...
2,Alto La Llama - Majada de Espineres,Piloña,43.410528294992,-5.284703713463256,"[Distancia:8,9kms, Dificultad:Baja, Altitud:99...",Si hay un lugar ligado en Asturias al caballo ...,Ruta,https://www.turismoasturias.es/descubre/natura...
3,Bosque de Peloño,Ponga,43.15642230739511,-5.12929198358272,"[Distancia:11,3kms, Dificultad:Media, Altitud:...",Entrar en Peloño es como hacerlo en un túnel d...,Ruta,https://www.turismoasturias.es/descubre/natura...
4,Bustio - Pendueles,Ribadedeva y Llanes,43.38712212798508,-4.577040542878124,"[Distancia:21,9kms, Dificultad:Alta, Código:GR...",Bustio - Pendueles21.9Se accede al punto de co...,Ruta,https://www.turismoasturias.es/descubre/natura...


In [20]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/routes.csv', index=False)

## PLAYAS

Dada la url de la página correspondiente a las playas de turismoasturias.es, se busca obtener un dataframe con las características de cada una de ellas. El primer paso es obtener las urls de cada una de las playas existentes con ayuda de la función obtener_enlaces_lugturistico_requests (117 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [21]:
# Extraer todas las url de las playas
start= time.time() # Contabilizar el tiempo
# Definir los parámetros de la función (url base y páginas de 1 a 10)
url_base = "https://www.turismoasturias.es/costa/playas?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u_delta=12&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_MTK2sQFT6y6u_cur="
pag_final = 10

# Aplicar la función para obtener los enlaces
enlaces_completo = obtener_enlaces_lugturistico_requests(url_base, pag_final)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.29min


In [23]:
# Extraer las características de los enlaces
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Playa")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 1.5min


In [24]:
df_resultados.head(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,Cala Salencia,Cudillero,+43.57245,-6.28010,"[Servicios:Admite animales, Longitud:150 m., T...",Territorio virgen con acceso para perros en cu...,Playa,https://www.turismoasturias.es/descubre/costa/...
1,Playa Arenal de Morís,Caravia,43.474690,-5.173349,[Servicios:Aparcamiento de pago | \n\t\t\t\t\t...,"Muy bien equipada, accesible, versátil y en fo...",Playa,https://www.turismoasturias.es/descubre/costa/...
2,Playa Carranques,Carreño,+43.57753,-5.74332,[Servicios:Aparcamiento gratuito | \n\t\t\t\t\...,"Un pasado como puerto ballenero, situado en la...",Playa,https://www.turismoasturias.es/descubre/costa/...
3,Playa Concha de Artedo,Cudillero,+43.56453,-6.18882,[Servicios:Servicio hostelería | \n\t\t\t\t\t\...,En forma de concha y con aguas cristalinasLa C...,Playa,https://www.turismoasturias.es/descubre/costa/...
4,Playa de Aguilar y Playa de Campofrío,Muros de Nalón y Cudillero,+43.55483,-6.11672,[Servicios:Playa sin Humo | \n\t\t\t\t\t\t\t\t...,Un horizonte rocoso idóneo para fondear barcos...,Playa,https://www.turismoasturias.es/descubre/costa/...


In [ ]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/beaches.csv', index=False)

## PUEBLOS

Los pueblos no aparecen ordenados de la misma manera que los casos anteriores, en este caso, aparecen dividos por zonas y estas, a su vez, divididas por itinerarios. Dentro de estos itinerarios es donde se pueden encontrar los pueblos. Además, hay pueblos que aparecen en otra zona, como "pueblos marineros", por lo que es necesario extraerlos también. Por lo tanto, lo primero es encontrar las urls correspondientes a cada uno de los pueblos, por un lado los correspondientes a cada itinerario y por otro los pueblos marineros (proceso que se hace por pasos y son 94 pueblos). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [26]:
# Extraer las urls de los pueblos
start= time.time() # Contabilizar el tiempo

## PASO 1: urls de las zonas, tienen la misma forma todas (acceder a la página requiere más tiempo)
url_0 = 'https://www.turismoasturias.es/turismo-rural/'
zonas = ["occidente", "centro", "oriente"]
urls_1 = [url_0 + zona for zona in zonas]

## PASO 2: extraer las urls de las subdivisiones de cada zona (los llama itinerarios y los numera a partir de 1 por zonas)
urls_2 = []
for url in urls_1:
    response = requests.get(url)  # Hacer la solicitud a la página
    if response.status_code != 200:
        print(f"Error al acceder a {url}")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    # Encontrar todos los elementos con clase 'card' (cada itinerario)
    itinerarios = soup.find_all(class_="card")
    # Construir las URLs de los itinerarios
    urls_2.extend([f"{url}/itinerario{i+1}" for i in range(len(itinerarios))])

## PASO 3: extraer la url correspondiente a cada pueblo
enlaces_completo = []
for url in urls_2:
    response = requests.get(url)  # Hacer la solicitud a la pagina
    if response.status_code != 200:
        print(f"Error al acceder a {url}")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    # Buscar los elementos que contienen los enlaces
    elementos = soup.find_all("a", class_="d-block image-wrapper aspect-ratio-bg-cover aspect-ratio-4-to-3")
    # Extraer las URLs 
    for enlace in elementos:
        if enlace.has_attr("href"):
            enlaces_completo.append(enlace["href"])

# Obtener los enlaces correspondientes a los pueblos marineros
url = "https://www.turismoasturias.es/costa/pueblos-marineros"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response = requests.get(url, headers=headers) 

# Extraer todos los enlaces de los pueblos marineros
enlaces_completo2 = obtener_enlaces_pagina_requests(response)

# Unir los enlaces de todos los pueblos encontrados
enlaces_pueblos = enlaces_completo + enlaces_completo2

print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.3min


In [28]:
# Obtener las características de cada enlace que corresponde a un pueblo
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_pueblos, "Pueblo")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 1.25min


In [29]:
# Eliminar los pueblos que aparecen duplicados (por aparecer en un itinerario y como pueblo marinero)
df_resultados = df_resultados.drop_duplicates(subset='Nombre', keep='first')

In [30]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
49,Laviana,Valle del Nalón,43.2448002,-5.5626496,[Dirección:Centro de Interpretación Armando Pa...,Lavianaostenta el honor de contar con uno de l...,Pueblo,https://www.turismoasturias.es/descubre/donde-...
50,Lena,Montaña Central,43.1591428,-5.8292909,"[Dirección:Esquina del Marqués de San Feliz, 2...","Vió pasar a losromanospor lavía Carisa, y más ...",Pueblo,https://www.turismoasturias.es/descubre/donde-...
6,Villanueva de Oscos,Oscos-Eo,43.3113918,-6.9864039,"[Dirección:Plaza del Ayuntamiento. 33777, Vila...","Como sus dos compañeros de comarca, el municip...",Pueblo,https://www.turismoasturias.es/descubre/donde-...
22,Tineo,Comarca Vaqueira,43.333851,-6.408835,"[Dirección:Plaza El Fontán, 22. 33870, Tinéu, ...","Es el segundo concejo más grande de Asturias, ...",Pueblo,https://www.turismoasturias.es/descubre/donde-...
88,Ortigueira,Coaña,43.560892,-6.740782,[Dirección:Oficina de Turismo: Castro de Coaña...,"A solo veinte metros de altitud sobre el mar, ...",Pueblo,https://www.turismoasturias.es/descubre/costa/...


In [31]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/towns.csv', index=False)

## PRERROMÁNICO

Dada la url de la página correspondiente al perromanico de turismoasturias.es, se busca obtener un dataframe con las características de cada una de ellas. El primer paso es obtener las urls de cada una de los lugares existentes con ayuda de la función obtener_enlaces_pagina_requests (15 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [32]:
# Extraer las urls del prerrománico
url = "https://www.turismoasturias.es/cultura/prerromanico"

start= time.time() # Contabilizar el tiempo
# Lista para guardar la url de cada monumento
response = requests.get(url)  # Hacer la solicitud HTTP
if response.status_code != 200:
    print(f"Error al acceder a {url}")
else:
    enlaces_completo = obtener_enlaces_pagina_requests(response)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.03min


In [34]:
# Obtener las características de cada enlace que corresponde al prerromanico
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Prerromanico")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.2min


In [35]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
9,La Foncalada,Oviedo,43.365280,-5.846036,"[Dirección:C/ Foncalada (esquina C/ Gascona), ...",La fuente de Foncalada data del siglo IX y fue...,Prerromanico,https://www.turismoasturias.es/descubre/cultur...
2,Iglesia de San Pedro de Nora,Las Regueras,43.36908,-5.962004,"[Titularidad:Eclesiástica, Teléfono:985 784 25...",La iglesia de San Pedro de Nora presenta grand...,Prerromanico,https://www.turismoasturias.es/descubre/cultur...
6,Iglesia de Santa María de Bendones,Oviedo,43.3362,-5.806374,"[Titularidad:Eclesiástica, Teléfono:985 942 36...","Iglesia prerrománica edificada, probablemente,...",Prerromanico,https://www.turismoasturias.es/descubre/cultur...
12,Santa Cristina de Lena,Lena,43.128594,-5.813222,"[Dirección:La Vega'l Rei, Titularidad:Eclesiás...","La iglesia de Santa Cristina de Lena, situada ...",Prerromanico,https://www.turismoasturias.es/descubre/cultur...
13,Santa María del Naranco,Oviedo,43.379055,-5.865849,[Dirección:Avda de los Monumentos (Monte Naran...,El Palacio de Ramiro I ubicado en la falda del...,Prerromanico,https://www.turismoasturias.es/descubre/cultur...


In [36]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/pre_romanesque.csv', index=False)

## ARTE RUPESTRE

Dada la url de la página correspondiente al arte rupestre de turismoasturias.es, se busca obtener un dataframe con las características de cada una de ellas. El primer paso es obtener las urls de cada una de los lugares existentes con ayuda de la función obtener_enlaces_pagina_requests (15 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.

In [37]:
# Extraer las urls del arte rupestre
url = "https://www.turismoasturias.es/cultura/arte-rupestre"

start= time.time() # Contabilizar el tiempo
# Lista para guardar la url de cada monumento
response = requests.get(url)  # Hacer la solicitud HTTP
if response.status_code != 200:
    print(f"Error al acceder a {url}")
else:
    enlaces_completo = obtener_enlaces_pagina_requests(response)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.04min


In [39]:
# Obtener las características de cada enlace que corresponde a arte rupestre
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Arte rupestre")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.2min


In [66]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
3,Cueva de El Buxu,Cangas de Onís,43.357730,-5.099614,"[Dirección:■33589 - Cardes, Teléfono:608 175 467]",La Cueva de El Buxu se encuentra situada en la...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...
5,Cueva de La Lluera,Oviedo,43.334687,-5.933297,"[Dirección:Priorio ■33174 - Les Caldes, Teléfo...",Su extraordinaria importancia se debe tanto a ...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...
10,Cueva del Conde o del Fornu,Santo Adriano,43.298253,-5.986963,"[Dirección:■33115 - Tuñon, Teléfono:985 76 14 ...",En este abrigo rocoso se descubrió abundante i...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...
14,La Cuevona de Ardines,Ribadesella,43.4607,-5.06735,[Dirección:Avda. de Tito Bustillo ■33560 - Rib...,Esta cueva tiene yacimiento arqueológico ycons...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...
0,Abrigo de Santo Adriano,Santo Adriano,43.298580,-5.987977,"[Dirección:■33115 - Tuñón, Teléfono:985 76 14 ...",Este abrigo alberga un importante conjunto de ...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...


In [40]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/rock_art.csv', index=False)

## PATRIMONIO RELIGIOSO

Dada la url de la página correspondiente al patrimonio religioso de turismoasturias.es, se busca obtener un dataframe con las características de cada una de ellas. El primer paso es obtener las urls de cada uno de los elemento de patrimonio religioso existentes con ayuda de la función obtener_enlaces_lugturistico_requests (56 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.
OJO: Covadonga tiene una página diferente a todo lo demás.

In [52]:
# Extraer todas las url de los museos y espacios culturales
start= time.time() # Contabilizar el tiempo
# Definir los parámetros de la función (url base y páginas de 1 a 13)
url_base = "https://www.turismoasturias.es/descubre/cultura/patrimonio-religioso?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_BeX7yFrckmNb&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_BeX7yFrckmNb_delta=12&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_BeX7yFrckmNb_cur="
pag_final = 5

# Aplicar la función para obtener los enlaces
enlaces_completo = obtener_enlaces_lugturistico_requests(url_base, pag_final)
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.09min


In [54]:
# Obtener las características de cada enlace que corresponde a una ruta de senderismo
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Patrimonio religioso")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

Error en:  https://www.turismoasturias.es/descubre/cultura/patrimonio-religioso/santuario-de-covadonga
finalizado en 0.67min


In [55]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
13,Monasterio de Santa María de Villamayor,Piloña,43.359592,-5.30599,"[Dirección:■33583 - Villamayor, Época:Románico]",La fundación de este monasterio de monjas bene...,Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...
51,San Esteban (Piantón),Vegadeo,43.460894,-7.028376,"[Dirección:■33778 - Piantón, Teléfono:985 634 ...",El antiguo monasterio de San Esteban de Piantó...,Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...
47,Colegiata de Santa María la Real de Tanes,Caso,43.204097,-5.395984,"[Dirección:■339994 - Tañes, Época:Siglo XVI]",La Colegiata de Santa María la Real de Tanes s...,Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...
41,Iglesia de Santiago (Pesoz),Pesoz,43.257690,-6.876326,"[Dirección:Bo. Pesoz, 46 ■33735 - Pesoz, Época...","El templo tiene su origen en el románico, prev...",Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...
14,Monasterio de Santa María de Villanueva de Oscos,Villanueva de Oscos,43.311592,-6.986192,[Dirección:■33776 - Vilanova],El monasterio de Santa María fue durante siglo...,Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...


In [56]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/religious_heritage.csv', index=False)

## NATURALEZA

Este apartado esta muy poco estructurado, por lo que requiere muchos pasos intermedios par obtener los enlaces de los que se sacan las caracteristicas para formar el dataframe.
Dada la url de la página correspondiente a la naturaleza de turismoasturias.es, se busca obtener un dataframe con las características de cada elemento. El primer paso es obtener las urls de las distintas subsecciones que tiene "naturaleza" (paso 1), para cada subsección hay que extraer las urls de los lugares turisticos, teniendo en cuenta que cada una de ellas tiene una estructura diferente (paso 2) ( 127 enlaces). Posteriormente, con la función obtener_datos_enlaces se obtienen las características de esos lugares.


In [57]:
# Extraer los enlaces de los diferentes lugares ralacionados con naturaleza
start= time.time() 
# Paso 1: extraer las urls de las secciones (No se puede usar la función porque la información no sigue la misma estructura que antes)
url = "https://www.turismoasturias.es/naturaleza"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response = requests.get(url, headers=headers) # Hacer la solicitud a la página
soup = BeautifulSoup(response.text, "html.parser") # Extraer el html

# Lista para almacenar las URLs
enlaces_pagina = []

# Obtener los elementos que contienen enlaces
elementos = soup.find_all(class_='card')
# Para cada elemento, se selecciona la URL correspondiente y se agrega a la lista
for enlace in elementos:
    if enlace.has_attr("href"):  # Verifica si tiene href
        enlaces_pagina.append(enlace["href"])  # Agrega el enlace a la lista

enlaces_pagina = enlaces_pagina[:5] # Nos quedamos con los 5 primeros, pues los demás incluyen varias categorias ya estudiadas

# Paso 2: para cada enlace anterior, obtener los enlaces de la correspondiente página (hay que ir por pasos)
enlaces_completo=[]

#################################################################################################
# Reservas de la biosfera (enlaces a los diferentes lugares directamente, pero no en el formato habitual, no se puede usar obtener_enlaces_pagina_requests())
url = enlaces_pagina[0]
response = requests.get(url, headers=headers) # Cargar el contenido de la página
soup = BeautifulSoup(response.text, "html.parser")
# Obtener el elemento que contiene el enlace
elementos = soup.find_all(class_='card')
for enlace in elementos:
    if enlace.has_attr("href"):  # Verifica si tiene href
        enlaces_completo.append(enlace["href"])  # Agrega el enlace a la lista

################################################################################################
# Reservas naturales y paisajes protegidos (enlaces a los diferentes lugares directamente)
for id in [1,2]:
    url = enlaces_pagina[id]
    response = requests.get(url, headers=headers) # Cargar el contenido de la página
    soup = BeautifulSoup(response.text, "html.parser")
    enlaces_completo.extend(obtener_enlaces_pagina_requests(response))

################################################################################################
# Monumentos naturales (Hay que completar la url, tiene varias páginas, se utiliza obtener_enlaces_lugturistico_requests)
# Definir los parámetros de la función (url base y páginas de 1 a 13)
url_base = "https://www.turismoasturias.es/naturaleza/monumentos-naturales?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_a5ZA4IMtVJvF&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_a5ZA4IMtVJvF_allCategoryIds=41531&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_a5ZA4IMtVJvF_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_a5ZA4IMtVJvF_cur="
pag_final = 3
# Aplicar la función para obtener los enlaces
enlaces_completo.extend(obtener_enlaces_lugturistico_requests(url_base, pag_final))

################################################################################################
# Otros espacios naturales (son enlaces, que tienen más enlaces, algunos con páginas)
enlaces_intermedios =[]
url = enlaces_pagina[4]
response = requests.get(url, headers=headers) # Cargar el contenido de la página
soup = BeautifulSoup(response.text, "html.parser")
# Obtener el elemento que contiene el enlace
elementos = soup.find_all(class_='card')
for enlace in elementos:
    if enlace.has_attr("href"):  # Verifica si tiene href
        enlaces_intermedios.append(enlace["href"])  # Agrega el enlace a la lista

# Extraer los enlaces individuales de los que no tienen páginas
for id2 in [0,1,2,3,5,6]:
    url = enlaces_intermedios[id2]
    response = requests.get(url, headers=headers) # Cargar el contenido de la página
    soup = BeautifulSoup(response.text, "html.parser")
    enlaces_completo.extend(obtener_enlaces_pagina_requests(response))

# Extraer los enlaces de los que contienen varias páginas
url_base = "https://www.turismoasturias.es/descubre/naturaleza/otros-espacios/areas-recreativas?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_Ny2Jn11AIsID&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_Ny2Jn11AIsID_allCategoryIds=41712&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_Ny2Jn11AIsID_delta=12&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_Ny2Jn11AIsID_cur="
pag_final = 2
# Aplicar la función para obtener los enlaces
enlaces_completo.extend(obtener_enlaces_lugturistico_requests(url_base, pag_final))

print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 0.28min


In [59]:
# Obtener las características de cada enlace que corresponde a naturaleza
start= time.time() # Contabilizar el tiempo
df_resultados = obtener_datos_enlaces(enlaces_completo, "Naturaleza")
print(f'finalizado en {round((time.time()-start)/60,2)}min') #Tiempo

finalizado en 1.53min


In [60]:
df_resultados.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
34,Playa de Gulpiyuri,Llanes,+43.44756,-4.886105,[],Gulpiyuri es una pequeña playa situada en posi...,Naturaleza,https://www.turismoasturias.es/descubre/natura...
83,Río Dobra,"Amieva, Cangas de Onís",43.251298,-5.048053,[Extensión:60 km2],"El Río Dobra con sus 23 Km de longitud, discur...",Naturaleza,https://www.turismoasturias.es/descubre/natura...
86,Río Caudal,"Aller, Lena, Mieres, Morcín, Riosa, Ribera de ...",43.267956,-5.800966,[Extensión:61 km2],"Nace en Santa Cruz de Mieres, fruto de la unió...",Naturaleza,https://www.turismoasturias.es/descubre/natura...
6,Parque Natural de Ponga,Ponga,+43.1938484,-5.1640605,"[Extensión:255 km2, Altitud Máxima:2.142 m. Pe...",El Parque Natural de Ponga fue declarado Reser...,Naturaleza,https://www.turismoasturias.es/descubre/natura...
16,Yacimientos de Icnitas,"Colunga, Gijón, Ribadesella y Villaviciosa",+43.500565,-5.259361,[],"Hace más de 150 millones de años, en el períod...",Naturaleza,https://www.turismoasturias.es/descubre/natura...


In [61]:
# Guardar el dataframe como csv
df_resultados.to_csv('../../Data/Tourist_places/nature.csv', index=False)

## COMPLETO

In [62]:
# Ruta donde se encuentran los archivos CSV
ruta_archivos = '../../Data/Tourist_places'

# Lista para almacenar los DataFrames de cada CSV
dfs = []

# Iterar sobre los archivos CSV en la carpeta
for archivo in os.listdir(ruta_archivos):
    if archivo.endswith('.csv'):
        # Cargar el archivo CSV en un DataFrame
        df = pd.read_csv(os.path.join(ruta_archivos, archivo))
        dfs.append(df)

# Unir todos los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)

In [63]:
df_final.sample(5)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
582,Senda del Agua,"Corvera de Asturias, Illas, Castrillón y Soto ...",43.527927,-5.992096,"['Distancia:24,9kms', 'Dificultad:Baja', 'Códi...","El 6 de marzo de 1965, un periódico nacional p...",Ruta,https://www.turismoasturias.es/descubre/natura...
468,Cueva de La Lluera,Oviedo,43.334687,-5.933297,"['Dirección:Priorio', 'Teléfono:674 800 385', ...",Su extraordinaria importancia se debe tanto a ...,Arte rupestre,https://www.turismoasturias.es/descubre/cultur...
19,Playa de Cambaredo,El Franco,43.556184,-6.817541,['Servicios:Playa sin Humo | \n\t\t\t\t\t\t\t\...,"Playa estrecha, alargada y pedregosa, con esca...",Playa,https://www.turismoasturias.es/descubre/costa/...
180,Colegiata de Nuestra Señora de Covadonga,Cangas de Onís,43.307926,-5.054678,['DóndeDóndeCentro de AsturiasOccidente de Ast...,La Colegiata de Covadonga está ubicada en una ...,Cultura,https://www.turismoasturias.es/cultura/museos-...
46,Playa de Monellos,El Franco,43.562510,-6.852036,"['Servicios:Playa sin Humo', 'Longitud:100 m.'...",Un paraíso escondido donde practicar la pesca ...,Playa,https://www.turismoasturias.es/descubre/costa/...


In [ ]:
# Eliminamos los lugares turísticos que esten repetidos (hay algunos que se consideran en varias categorias)
df_final = df_final.drop_duplicates(subset='Nombre', keep='first')
df_final.shape # 674 lugares tiurísticos

(674, 8)

In [65]:
# Recuento de lugares turisticos de cada tipo
df_final["Tipo_lugar_turistico"].value_counts()

Tipo_lugar_turistico
Cultura                 150
Ruta                    118
Naturaleza              117
Playa                   114
Pueblo                   90
Patrimonio religioso     55
Prerromanico             15
Arte rupestre            15
Name: count, dtype: int64

In [66]:
# Guardar el dataframe como csv
df_final.to_csv('../../Data/Data_used/tourist_places_data.csv', index=False)